In [ ]:
pip install pandas numpy

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
orders = pd.read_csv("/content/orders.csv")
products = pd.read_csv("/content/products.csv")
order_products_train = pd.read_csv("/content/order_products__train.csv")
order_products_prior = pd.read_csv("/content/order_products__prior.csv")
aisles = pd.read_csv("/content/aisles.csv")
departments = pd.read_csv("/content/departments.csv")

In [ ]:
cols_to_convert = ['order_number', 'order_dow', 'order_hour_of_day']

for col in cols_to_convert:
    orders[col] = orders[col].astype('Int64')

In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
cols_to_convert = ['product_id', 'add_to_cart_order', 'reordered']

for col in cols_to_convert:
    order_products_train[col] = order_products_train[col].astype('Int64')

In [ ]:
cols_to_convert = ['add_to_cart_order', 'reordered']

for col in cols_to_convert:
    order_products_prior[col] = order_products_prior[col].astype('Int64')

In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
prior_df = order_products_prior.merge(orders, on ='order_id', how='inner')
prior_df = prior_df.merge(products, on = 'product_id', how = 'left')
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


In [ ]:
prior_df['user_product_reorder_count'] = prior_df.groupby(['user_id', 'product_id']).cumcount() + 1
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_product_reorder_count
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


# User-based features

In [ ]:
prior_df['days_since_prior_order'] = prior_df['days_since_prior_order'].fillna(0)

user_features = prior_df.groupby('user_id').agg(
    avg_days_between_orders=('days_since_prior_order', 'mean'),
    total_orders=('order_number', 'max')
).reset_index()

In [ ]:
user_features.head()

,user_id,avg_days_between_orders,total_orders
0,1,18.542373,10
1,2,14.902564,14
2,3,10.181818,12
3,4,11.944444,5
4,5,10.189189,4


# Product-based features

In [ ]:
product_features = prior_df.groupby('product_id').agg(
    reorder_ratio=('reordered', 'mean'),
    times_reordered=('reordered', 'sum')
).reset_index()

In [ ]:
product_features = product_features.merge(products, on = 'product_id', how = 'left')

In [ ]:
product_features.drop(['product_name', 'aisle_id', 'department_id'], axis = 1, inplace = True)

In [ ]:
product_features.head()

,product_id,reorder_ratio,times_reordered
0,1,0.613391,1136
1,2,0.133333,12
2,3,0.732852,203
3,4,0.446809,147
4,5,0.6,9


# User-product interaction features

In [ ]:
user_product_features = prior_df.groupby(['user_id', 'product_id']).agg(
    user_product_reorder_count=('reordered', 'sum')
).reset_index()

In [ ]:
user_product_features['user_id']

,user_id
0,1
1,1
2,1
3,1
4,1
...,...
13307948,206209
13307949,206209
13307950,206209
13307951,206209


In [ ]:
user_product_features.head()

,user_id,product_id,user_product_reorder_count
0,1,196,9
1,1,10258,8
2,1,10326,0
3,1,12427,9
4,1,13032,2


In [ ]:
user_features.isnull().sum()

,0
user_id,0
avg_days_between_orders,0
total_orders,0


In [ ]:
user_features['avg_days_between_orders'].nunique()

102702

In [ ]:
user_features['avg_days_between_orders']

,avg_days_between_orders
0,18.542373
1,14.902564
2,10.181818
3,11.944444
4,10.189189
...,...
206204,9.687500
206205,3.985965
206206,13.278027
206207,7.310192


In [ ]:
user_features.to_pickle('user_features.pkl')
product_features.to_pickle('product_features.pkl')
user_product_features.to_pickle('user_product_features.pkl')

In [ ]:
user_features=pd.read_pickle('user_features.pkl')
product_features=pd.read_pickle('product_features.pkl')
user_product_features=pd.read_pickle('user_product_features.pkl')

In [ ]:
user_product_features

,user_id,product_id,user_product_reorder_count
0,1,196,9
1,1,10258,8
2,1,10326,0
3,1,12427,9
4,1,13032,2
...,...,...,...
13307948,206209,43961,2
13307949,206209,44325,0
13307950,206209,48370,0
13307951,206209,48697,0


In [ ]:
user_product_features['user_product_reorder_count'].unique()

<IntegerArray>
[ 9,  8,  0,  2,  1,  7,  5,  3,  4,  6, 10, 11, 12, 32, 24, 29, 16, 20, 17,
 22, 19, 15, 14, 13, 33, 52, 18, 27, 21, 23, 43, 56, 25, 28, 30, 46, 35, 78,
 66, 39, 44, 34, 41, 31, 38, 26, 40, 45, 36, 47, 62, 48, 37, 49, 54, 50, 42,
 51, 83, 67, 61, 53, 60, 73, 58, 57, 69, 76, 70, 80, 59, 63, 68, 55, 71, 90,
 84, 87, 79, 75, 64, 88, 74, 85, 82, 86, 72, 65, 77, 81, 98, 89, 91, 94, 92,
 97, 95, 93, 96]
Length: 99, dtype: Int64

In [ ]:
product_features

,product_id,reorder_ratio,times_reordered
0,1,0.613391,1136
1,2,0.133333,12
2,3,0.732852,203
3,4,0.446809,147
4,5,0.6,9
...,...,...,...
49672,49684,0.111111,1
49673,49685,0.122449,6
49674,49686,0.7,84
49675,49687,0.461538,6


In [ ]:
user_features

,user_id,avg_days_between_orders,total_orders
0,1,18.542373,10
1,2,14.902564,14
2,3,10.181818,12
3,4,11.944444,5
4,5,10.189189,4
...,...,...,...
206204,206205,9.687500,3
206205,206206,3.985965,67
206206,206207,13.278027,16
206207,206208,7.310192,49
